In [54]:
# Importo las librerías que vimos en la semana 4

import os
import io
import sys
import json
import requests
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

In [55]:
import google.generativeai as genai

In [56]:
# environment

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')
#Agrego google 
os.environ['GOOGLE_API_KEY']=os.getenv('GOOGLE_API_KEY','your-key-if-not-using-env')


In [57]:
#Modelos
openai = OpenAI()
claude = anthropic.Anthropic()

OPENAI_MODEL = "gpt-4o"
CLAUDE_MODEL = "claude-3-5-sonnet-20240620"


In [62]:
genai.configure()
GEMINI_MODEL= "gemini-1.5-flash"

In [40]:
#Mantengo el mismo system message ya q solo agrego Gemini
system_message = "Eres un asistente que reimplementa código Python en C++ de alto rendimiento para una Mac M4. "
system_message += "Responde solo con código C++; usa los comentarios con moderación y no proporciones ninguna explicación más allá de comentarios ocasionales. "
system_message += "La respuesta C++ debe producir una salida idéntica en el menor tiempo posible."

In [41]:
#Mantengo el mismo user prompt ya q solo agrego Gemini
def user_prompt_for(python):
    user_prompt = "Reescribe este código Python en C++ con la implementación más rápida posible que produzca una salida idéntica en el menor tiempo posible."
    user_prompt += "Responde solo con código C++; no expliques tu trabajo más allá de algunos comentarios."
    user_prompt += "Manten la implementación de la generación de números aleatorios idénticos para que los resultados de la coincidencia sean exactos."
    user_prompt += "Responde solo con código C++; no añadas nada más que código; usa los comentarios con moderación y no proporciones ninguna explicación más allá de comentarios ocasionales. "
    user_prompt += "Presta atención a los tipos de números para asegurar que no haya desbordamientos de int (overflow). Recuerda incluir todos los paquetes de C++ necesarios, como iomanip.\n\n"    
    user_prompt += python
    return user_prompt

In [42]:
def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [43]:
def write_output(cpp):
    code = cpp.replace("```cpp","").replace("```","")
    with open("optimized.cpp", "w") as f:
        f.write(code)

In [44]:
def optimize_gpt(python):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end='', flush=True)
    write_output(reply)

In [45]:
def optimize_claude(python):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            print(text, end="", flush=True)
    write_output(reply)

In [63]:
#Sumo el chunk de Gemini usando gemini-1.5-flash
import google.generativeai as genai

def optimize_gemini(python):
    stream = genai.GenerativeModel("gemini-1.5-flash").generate_content(user_prompt_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.text or ""
        reply += fragment
        print(fragment, end='', flush=True)
    write_output(reply)



In [64]:
#uso el mismo chunk de codigo
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [65]:
#pruebo la funcion de gemini
optimize_gemini(pi)

```cpp
#include <iostream>
#include <chrono>
 <iomanip>

//Usar double para evitar desbordamientos
 std; namespace
using namespace chrono;


double calculate(long long iterations, long long param1, long long param2) {
1.0;double result = 
    for (long long i = 1; i <= iterations; ++i) {
)i * param1 - param2; = (long double
        result -= (1.0/j);
        j = (long double)i * param1 + param2;
        result += (1.0/j);
    }
    return result;
}

int main() {
    auto start_time = high_resolution_clock::now();

, 1) * 4;e result = calculate(100000000, 4

    auto end_time = high_resolution_clock::now();
    auto duration = duration_cast<microseconds>(end_time - start_time);

;   cout << "Result: " << fixed << setprecision(12) << result << endl
    cout << "Execution Time: " << (double)duration.count() / 1000000.0 << " seconds" << endl;

    return 0;
}
```

In [66]:
#mantengo el python hard

python_hard = """
def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [39]:
optimize_gpt(python_hard)

```cpp
#include <iostream>
#include <vector>
#include <ctime>
#include <cstdint>
#include <limits>

 Linear Congruential Generator (LCG)
class LCG {
public:
    LCG(uint32_t seed, uint32_t a = 1664525, uint32_t c = 1013904223, uint32_t m = 0xFFFFFFFF)
        : value(seed), a(a), c(c), m(m) {}

    uint32_t next() {
        value = (static_cast<uint64_t>(a) * value + c) % m;
        return value;
    }

private:
    uint32_t value;
    const uint32_t a, c, m;
};

 Maximum subarray sum calculation
int max_subarray_sum(int n, uint32_t seed, int min_val, int max_val) {
    LCG lcg(seed);
    std::vector<int> random_numbers(n);
    
    for (int i = 0; i < n; ++i) {
        random_numbers[i] = static_cast<int>(lcg.next() % (max_val - min_val + 1)) + min_val;
    }

    int max_sum = std::numeric_limits<int>::min();
    for (int i = 0; i < n; ++i) {
        int current_sum = 0;
        for (int j = i; j < n; ++j) {
            current_sum += random_numbers[j];
            if (current_sum > 

In [67]:
#pruebo la funcon de gemini
optimize_gemini(python_hard)

```cpp
#include <iostream>
#include <vector>
 <limits> // Required for numeric_limits
 Required for high-resolution clock

// Use unsigned long long to avoid overflow
typedef unsigned long long ull;

 = 1664525, ull c = 1013904223, ull m = (1ULL << 32)) {
    return (a * seed + c) % m;
}

 {ng long max_subarray_sum(int n, ull seed, long long min_val, long long max_val)
    std::vector<long long> random_numbers(n);
ULL << 32); 1664525, c = 1013904223, m = (1
    for (int i = 0; i < n; ++i) {
        seed = lcg(seed, a, c, m);
_val;   random_numbers[i] = seed % (max_val - min_val + 1) + min
    }

    long long max_sum = std::numeric_limits<long long>::min();
    for (int i = 0; i < n; ++i) {
        long long current_sum = 0;
 < n; ++j) {(int j = i; j
            current_sum += random_numbers[j];
            if (current_sum > max_sum) {
                max_sum = current_sum;
            }
        }
    }
    return max_sum;
}

_max_subarray_sum(int n, ull initial_seed, long long min_val

In [68]:
def stream_gpt(python):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```cpp\n','').replace('```','')

In [69]:
def stream_claude(python):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply.replace('```cpp\n','').replace('```','')

In [70]:
## creo la funcion de streaming
def stream_gemini(python):
    stream = genai.GenerativeModel("gemini-1.5-flash").generate_content(user_prompt_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.text or ""
        reply += fragment
        yield reply.replace('```cpp\n','').replace('```','')

In [71]:
#funcion para luego usar gradio
def optimize(python, model):
    if model=="GPT":
        result = stream_gpt(python)
    elif model=="Claude":
        result = stream_claude(python)
    elif model=="Gemini":
        result = stream_gemini(python)
    else:
        raise ValueError("Modelo Desconocido")
    for stream_so_far in result:
        yield stream_so_far        

In [72]:
#Agrego Gemini a la opción del desplegable de gradio
with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Código en Python:", lines=10, value=python_hard)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude","Gemini"], label="Selecciona el modelo", value="GPT")
        convert = gr.Button("Convertir código")

    convert.click(optimize, inputs=[python, model], outputs=[cpp])

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


In [73]:
def execute_python(code):
        try:
            output = io.StringIO()
            sys.stdout = output
            exec(code)
        finally:
            sys.stdout = sys.__stdout__
        return output.getvalue()

In [74]:
def execute_cpp(code):
        write_output(code)
        try:
            compile_cmd = ["clang++", "-Ofast", "-std=c++17", "-march=armv8.5-a", "-mtune=apple-m2", "-mcpu=apple-m2", "-o", "optimized", "optimized.cpp"]
            compile_result = subprocess.run(compile_cmd, check=True, text=True, capture_output=True)
            run_cmd = ["./optimized"]
            run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
            return run_result.stdout
        except subprocess.CalledProcessError as e:
            return f"An error occurred:\n{e.stderr}"

In [75]:
css = """
.python {background-color: #306998;}
.cpp {background-color: #050;}
"""

In [76]:
#Agreego gemini a la segunda versionando de gradio
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convierte código de Python a C++")
    with gr.Row():
        python = gr.Textbox(label="Código en Python:", value=python_hard, lines=10)
        cpp = gr.Textbox(label="Código en C++:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude","Gemini"], label="Selecciona el modelo", value="GPT")
    with gr.Row():
        convert = gr.Button("Convertir el código")
    with gr.Row():
        python_run = gr.Button("Ejecutar Python")
        cpp_run = gr.Button("Ejecutar C++")
    with gr.Row():
        python_out = gr.TextArea(label="Resultado en Python:", elem_classes=["python"])
        cpp_out = gr.TextArea(label="Resultado en C++:", elem_classes=["cpp"])

    convert.click(optimize, inputs=[python, model], outputs=[cpp])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


In [79]:
def add_docstrings_and_comments(python_code):
    """
    Toma un código en Python y agrega automáticamente docstrings y comentarios explicativos.
    """
    prompt = f"""
    Agrega docstrings y comentarios explicativos al siguiente código de Python:
    
    {python_code}
    """
    model = genai.GenerativeModel("gemini-1.5-flash")
    response = model.generate_content(prompt)
    return response.text.replace("\n", "\n\n")


In [80]:
add_docstrings_and_comments(pi)

'```python\n\nimport time\n\n\n\ndef calculate(iterations, param1, param2):\n\n    """\n\n    Calcula una aproximación a un valor numérico utilizando una serie.\n\n\n\n    Args:\n\n        iterations (int): Número de iteraciones para la aproximación.  Debe ser un entero positivo.\n\n        param1 (float): Primer parámetro de la fórmula.\n\n        param2 (float): Segundo parámetro de la fórmula.\n\n\n\n    Returns:\n\n        float: El resultado aproximado de la serie.  Devuelve 0 si `iterations` es 0 o menor.\n\n\n\n    Raises:\n\n        ZeroDivisionError: Si `param1` y `param2` producen un denominador igual a cero en la fórmula.\n\n    """\n\n    if iterations <=0:\n\n        return 0\n\n    result = 1.0\n\n    for i in range(1, iterations+1):\n\n        # Calcula el primer término de la serie\n\n        j = i * param1 - param2\n\n        if j == 0:\n\n            raise ZeroDivisionError("Denominador igual a cero en la primera parte de la fórmula")\n\n        result -= (1/j)  # Res

In [81]:
#Crear una herramienta de codigo que agrega automáticamente docstrings y comentarios

system_message = "Eres un asistente que analiza y modifica código fuente de alto rendimiento para una Mac M4. "
system_message += "Procesa ese codigo, analiza la estructura del código (funciones, clases, módulos, variables).Identifica los propósitos y agrega explicaciones adecuadas. "
system_message += "Inserta docstrings y comentarios donde sean necesarios. "
system_message += "Se devuelve el mismo código pero con documentación añadida. en el menor tiempo posible."


In [82]:
def user_prompt_for(codigo):
    user_prompt = (
        "Optimiza el siguiente código para que se ejecute en el menor tiempo posible, "
        "manteniendo su salida idéntica. Luego, agrega docstrings y comentarios explicativos.\n\n"
        "Código:\n\n"
    )
    user_prompt += codigo
    return user_prompt

In [83]:
def messages_for(codigo):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(codigo)}
    ]

In [87]:
def explicar_gpt(codigo):

    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(codigo), stream=True)
    
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end='', flush=True)

In [85]:

ejemplo = """

def sum(param1, param2):
    return param1 + param2
"""

In [86]:
explicar_gpt(ejemplo)

El código dado es una función sencilla que suma dos números. Es eficiente y no requiere optimización adicional para mejorar el rendimiento, ya que se trata de una simple operación de suma. Sin embargo, puedo agregar documentación para aclarar su propósito:

```python
def sum(param1, param2):
    """
    Suma dos números.

    Esta función recibe dos parámetros numéricos y devuelve su suma. 
    Se asume que los parámetros proporcionados son valores numéricos (int, float).

    Parameters:
    param1: int or float
        El primer número a sumar.
    param2: int or float
        El segundo número a sumar.

    Returns:
    int or float
        La suma de param1 y param2.
    """
    return param1 + param2
```

### Explicaciones añadidas:
- **Docstring:** Se ha añadido un docstring para describir la función, explicando qué hace, qué parámetros recibe, y qué devuelve.
- **Comentarios internos:** No son necesarios debido a la simplicidad de la función, ya que el código es autoexplicativo.

'El código dado es una función sencilla que suma dos números. Es eficiente y no requiere optimización adicional para mejorar el rendimiento, ya que se trata de una simple operación de suma. Sin embargo, puedo agregar documentación para aclarar su propósito:\n\n```python\ndef sum(param1, param2):\n    """\n    Suma dos números.\n\n    Esta función recibe dos parámetros numéricos y devuelve su suma. \n    Se asume que los parámetros proporcionados son valores numéricos (int, float).\n\n    Parameters:\n    param1: int or float\n        El primer número a sumar.\n    param2: int or float\n        El segundo número a sumar.\n\n    Returns:\n    int or float\n        La suma de param1 y param2.\n    """\n    return param1 + param2\n```\n\n### Explicaciones añadidas:\n- **Docstring:** Se ha añadido un docstring para describir la función, explicando qué hace, qué parámetros recibe, y qué devuelve.\n- **Comentarios internos:** No son necesarios debido a la simplicidad de la función, ya que el 

In [52]:
import gradio as gr

def procesar_codigo(codigo, modelo):
    if modelo == "GPT":
        return explicar_gpt(codigo)
    return "Modelo no soportado"

# Definir la interfaz con Gradio
def crear_interfaz():
    with gr.Blocks() as interfaz:
        gr.Markdown("# Explicador de Código")

        codigo_input = gr.Textbox(label="Ingresa por favor tu código", lines=6, placeholder="Pega tu código aquí...")

        modelo_input = gr.Dropdown(
            choices=["GPT"], 
            value="GPT", 
            label="Selecciona un modelo"
        )
        boton = gr.Button("Explicar Código")
        salida = gr.Textbox(label="Explicación", lines=6, interactive=False)
        boton.click(procesar_codigo, inputs=[codigo_input, modelo_input], outputs=salida)

    interfaz.launch()

crear_interfaz()




* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


```cpp
#include <iostream>

 Función que resta dos enteros y devuelve el resultado.
int resta(int param1, int param2) {
    return param1 - param2;
}

int main() {
    int resultado = resta(10, 5); // Ejemplo de llamada a la función
    std::cout << resultado << std::endl; // Salida: 5
    return 0;
}
```